## Imports

In [ ]:
# Keep track of where the repo root is. Change this if the directory structure changes.
REPO_ROOT = "../"

from sklearn.preprocessing import FunctionTransformer
from math import nan
import pandas as pd

Update the following imports when new feature engineering functions and modules are defined:

In [2]:
from edit_text import _word_counter, _comment_empty
from other_features import _is_IP

In [3]:
train = pd.read_csv(REPO_ROOT + "Data/train.csv")

## Looking at dataset

In [4]:
train

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,num_recent_edits,num_recent_reversions,current_minor,current_timestamp,added_lines,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person
0,change,329595189,NaN,Nryan30,66,13,0,1259891940,219.78.124.42,NaN,...,0,0,False,1259894598,",==Leadership Through Emotion==,""Leadership is...",1259856305,NaN,False,1,0
1,change,232199357,/* Penis */,89.242.200.212,4,2,2,20080815230001,66.75.235.255,NaN,...,0,0,False,1218841201,It's fucking big.,1218816231,"""Falcon gained media attention after appearing...",True,4,1
2,change,329877752,Reverted edits by [[Special:Contributions/71.2...,Chamal N,18697,0,2,1208605428,71.208.113.72,NaN,...,2,1,True,1260025124,"""Japanese modern drama in the early twentieth ...",1260025104,""".nmbhgsdj;kfhds;akjfhds;fkjhsf;kjdshf;sdkjhfd...",False,3,0
3,change,253129486,NaN,Animaldudeyay1009,3,1,2,1227241317,J.delanoy,NaN,...,0,0,False,1227241840,I LIKE CHEESE. CHEESE IS GOOD. CHOCOLATE MILK ...,1227241120,"""A '''kaleidoscope''' is a tube of [[mirror]]s...",True,2,0
4,change,394520551,Adding Persondata using [[Project:AWB|AWB]] (7...,RjwilmsiBot,1602950,1309238,0,1257977968,LobãoV,NaN,...,0,0,True,1288757547,{{Persondata <!-- Metadata: see [[Wikipedia:Pe...,1285262356,",",False,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25483,change,327192485,http://fr.wikipedia.org/wiki/Championnat_de_Gu...,91.210.204.254,3,2,0,20091121232112,SmackBot,NaN,...,0,0,False,1258845672,BAD REQUEST,1251868453,BAD REQUEST,False,0,0
25484,change,234810735,/* History */,59.180.151.222,1,1,0,20080828164140,68.50.79.137,NaN,...,0,0,False,1219941700,"""In the past, droughts have periodically led t...",1219280292,"""In the past, droughts have periodically led t...",True,0,0
25485,change,329132348,/* In Tamil Nadu */,66.184.61.179,1,1,0,20091201230141,RJFJR,NaN,...,0,0,False,1259708501,"""[[Tamil Nadu]] is home to one of the oldest a...",1259002677,"""[[Tamil Nadu]] is home to one of the oldest a...",False,0,0
25486,change,240599711,/* Biography */,75.157.130.175,6,1,2,20080924030549,J.delanoy,NaN,...,2,1,False,1222225549,Yo mama,1222225537,"== Biography ==,""Paulo Coelho was born in [[Ri...",True,11,1


There are certain rows where the API requests for `added_lines` and `deleted_lines` failed. Instead of the pandas standard NaN value, they contain the value "BAD REQUEST". It could be useful to convert this missing value to the standard NaN representation (for models where we don't use `added_lines` and `deleted_lines`, perhaps)

In [5]:
bad_request_replacer = FunctionTransformer(
    lambda X: X.replace(to_replace="BAD REQUEST", value=nan, inplace=False)
)

In [6]:
train_no_na = bad_request_replacer.fit_transform(train)

In [7]:
train[
    (train["added_lines"] == "BAD REQUEST") | (train["deleted_lines"] == "BAD REQUEST")
]

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,num_recent_edits,num_recent_reversions,current_minor,current_timestamp,added_lines,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person
412,change,243900559,[[WP:AES|←]]Replaced content with 'This inform...,206.40.172.98,257,141,18,20081008145559,Yakimonoqueen,NaN,...,0,0,False,1223477759,BAD REQUEST,1216689170,BAD REQUEST,True,0,0
583,change,327495176,/* Contents */,Rich jj,10748,0,0,1206502467,Rich jj,NaN,...,0,0,False,1258994339,BAD REQUEST,1253733576,BAD REQUEST,False,0,0
903,change,328233914,/* Other nations */,202.191.126.50,94,38,0,20091127171653,202.191.126.50,NaN,...,0,0,False,1259342213,BAD REQUEST,1259341048,BAD REQUEST,False,0,0
1219,change,327688529,NaN,59.66.33.219,9,1,0,20091124163904,59.66.33.219,NaN,...,0,0,False,1259080744,BAD REQUEST,1259080650,BAD REQUEST,False,0,0
3836,change,327361172,/* Notable alumni */,76.119.181.97,3,2,0,20091122222431,144.212.3.4,NaN,...,0,0,False,1258928671,BAD REQUEST,1257799437,BAD REQUEST,False,0,0
3906,change,327060368,/* Community Site */,117.18.231.14,191,110,1,20091121050042,117.18.231.14,NaN,...,0,0,False,1258779642,BAD REQUEST,1258779546,BAD REQUEST,False,11,0
4103,change,234691044,/* Business interests */,88.110.164.1,18,8,4,20080828011122,J.delanoy,NaN,...,0,0,False,1219885882,BAD REQUEST,1219885856,BAD REQUEST,True,5,1
4177,change,234354286,/* Facilities */,60.50.88.203,3,1,2,20080826140930,Rror,NaN,...,0,0,False,1219759770,BAD REQUEST,1219759661,BAD REQUEST,True,0,0
4682,change,329767313,/* Skintight spacesuits */,69.211.106.77,4,3,0,20091204234825,Biglovinb,NaN,...,0,0,False,1259970505,BAD REQUEST,1258765532,BAD REQUEST,False,0,0
4715,change,327274489,/* Season 1 */,187.38.48.36,3,1,0,20091122114850,187.38.48.36,NaN,...,0,0,False,1258890530,BAD REQUEST,1258889412,BAD REQUEST,True,0,0


In [8]:
train_no_na[
    (train_no_na["added_lines"] == "BAD REQUEST")
    | (train_no_na["deleted_lines"] == "BAD REQUEST")
]

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,num_recent_edits,num_recent_reversions,current_minor,current_timestamp,added_lines,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person


In [9]:
train.head()

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,num_recent_edits,num_recent_reversions,current_minor,current_timestamp,added_lines,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person
0,change,329595189,NaN,Nryan30,66,13,0,1259891940,219.78.124.42,NaN,...,0,0,False,1259894598,",==Leadership Through Emotion==,""Leadership is...",1259856305,NaN,False,1,0
1,change,232199357,/* Penis */,89.242.200.212,4,2,2,20080815230001,66.75.235.255,NaN,...,0,0,False,1218841201,It's fucking big.,1218816231,"""Falcon gained media attention after appearing...",True,4,1
2,change,329877752,Reverted edits by [[Special:Contributions/71.2...,Chamal N,18697,0,2,1208605428,71.208.113.72,NaN,...,2,1,True,1260025124,"""Japanese modern drama in the early twentieth ...",1260025104,""".nmbhgsdj;kfhds;akjfhds;fkjhsf;kjdshf;sdkjhfd...",False,3,0
3,change,253129486,NaN,Animaldudeyay1009,3,1,2,1227241317,J.delanoy,NaN,...,0,0,False,1227241840,I LIKE CHEESE. CHEESE IS GOOD. CHOCOLATE MILK ...,1227241120,"""A '''kaleidoscope''' is a tube of [[mirror]]s...",True,2,0
4,change,394520551,Adding Persondata using [[Project:AWB|AWB]] (7...,RjwilmsiBot,1602950,1309238,0,1257977968,LobãoV,NaN,...,0,0,True,1288757547,{{Persondata <!-- Metadata: see [[Wikipedia:Pe...,1285262356,",",False,0,1


In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25488 entries, 0 to 25487
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   EditType               25488 non-null  object 
 1   EditID                 25488 non-null  int64  
 2   comment                18076 non-null  object 
 3   user                   25488 non-null  object 
 4   user_edit_count        25488 non-null  int64  
 5   user_distinct_pages    25488 non-null  int64  
 6   user_warns             25488 non-null  int64  
 7   user_reg_time          25488 non-null  int64  
 8   prev_user              25488 non-null  object 
 9   common                 0 non-null      float64
 10  current                0 non-null      float64
 11  previous               0 non-null      float64
 12  page_made_time         25488 non-null  int64  
 13  title                  25488 non-null  object 
 14  namespace              25488 non-null  object 
 15  cr

In [11]:
train.columns

Index(['EditType', 'EditID', 'comment', 'user', 'user_edit_count',
       'user_distinct_pages', 'user_warns', 'user_reg_time', 'prev_user',
       'common', 'current', 'previous', 'page_made_time', 'title', 'namespace',
       'creator', 'num_recent_edits', 'num_recent_reversions', 'current_minor',
       'current_timestamp', 'added_lines', 'previous_timestamp',
       'deleted_lines', 'isvandalism', 'num_edits_5d_before', 'is_person'],
      dtype='object')

In [12]:
word_counter = FunctionTransformer(_word_counter)
comment_empty = FunctionTransformer(_comment_empty)

In [13]:
train

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,num_recent_edits,num_recent_reversions,current_minor,current_timestamp,added_lines,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person
0,change,329595189,NaN,Nryan30,66,13,0,1259891940,219.78.124.42,NaN,...,0,0,False,1259894598,",==Leadership Through Emotion==,""Leadership is...",1259856305,NaN,False,1,0
1,change,232199357,/* Penis */,89.242.200.212,4,2,2,20080815230001,66.75.235.255,NaN,...,0,0,False,1218841201,It's fucking big.,1218816231,"""Falcon gained media attention after appearing...",True,4,1
2,change,329877752,Reverted edits by [[Special:Contributions/71.2...,Chamal N,18697,0,2,1208605428,71.208.113.72,NaN,...,2,1,True,1260025124,"""Japanese modern drama in the early twentieth ...",1260025104,""".nmbhgsdj;kfhds;akjfhds;fkjhsf;kjdshf;sdkjhfd...",False,3,0
3,change,253129486,NaN,Animaldudeyay1009,3,1,2,1227241317,J.delanoy,NaN,...,0,0,False,1227241840,I LIKE CHEESE. CHEESE IS GOOD. CHOCOLATE MILK ...,1227241120,"""A '''kaleidoscope''' is a tube of [[mirror]]s...",True,2,0
4,change,394520551,Adding Persondata using [[Project:AWB|AWB]] (7...,RjwilmsiBot,1602950,1309238,0,1257977968,LobãoV,NaN,...,0,0,True,1288757547,{{Persondata <!-- Metadata: see [[Wikipedia:Pe...,1285262356,",",False,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25483,change,327192485,http://fr.wikipedia.org/wiki/Championnat_de_Gu...,91.210.204.254,3,2,0,20091121232112,SmackBot,NaN,...,0,0,False,1258845672,BAD REQUEST,1251868453,BAD REQUEST,False,0,0
25484,change,234810735,/* History */,59.180.151.222,1,1,0,20080828164140,68.50.79.137,NaN,...,0,0,False,1219941700,"""In the past, droughts have periodically led t...",1219280292,"""In the past, droughts have periodically led t...",True,0,0
25485,change,329132348,/* In Tamil Nadu */,66.184.61.179,1,1,0,20091201230141,RJFJR,NaN,...,0,0,False,1259708501,"""[[Tamil Nadu]] is home to one of the oldest a...",1259002677,"""[[Tamil Nadu]] is home to one of the oldest a...",False,0,0
25486,change,240599711,/* Biography */,75.157.130.175,6,1,2,20080924030549,J.delanoy,NaN,...,2,1,False,1222225549,Yo mama,1222225537,"== Biography ==,""Paulo Coelho was born in [[Ri...",True,11,1


In [14]:
word_counter.fit_transform(train)

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,current_minor,current_timestamp,added_lines,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person,added_word_count,deleted_word_count
0,change,329595189,NaN,Nryan30,66,13,0,1259891940,219.78.124.42,NaN,...,False,1259894598,",==Leadership Through Emotion==,""Leadership is...",1259856305,NaN,False,1,0,122,0
1,change,232199357,/* Penis */,89.242.200.212,4,2,2,20080815230001,66.75.235.255,NaN,...,False,1218841201,It's fucking big.,1218816231,"""Falcon gained media attention after appearing...",True,4,1,3,175
2,change,329877752,Reverted edits by [[Special:Contributions/71.2...,Chamal N,18697,0,2,1208605428,71.208.113.72,NaN,...,True,1260025124,"""Japanese modern drama in the early twentieth ...",1260025104,""".nmbhgsdj;kfhds;akjfhds;fkjhsf;kjdshf;sdkjhfd...",False,3,0,33,33
3,change,253129486,NaN,Animaldudeyay1009,3,1,2,1227241317,J.delanoy,NaN,...,False,1227241840,I LIKE CHEESE. CHEESE IS GOOD. CHOCOLATE MILK ...,1227241120,"""A '''kaleidoscope''' is a tube of [[mirror]]s...",True,2,0,89,603
4,change,394520551,Adding Persondata using [[Project:AWB|AWB]] (7...,RjwilmsiBot,1602950,1309238,0,1257977968,LobãoV,NaN,...,True,1288757547,{{Persondata <!-- Metadata: see [[Wikipedia:Pe...,1285262356,",",False,0,1,42,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25483,change,327192485,http://fr.wikipedia.org/wiki/Championnat_de_Gu...,91.210.204.254,3,2,0,20091121232112,SmackBot,NaN,...,False,1258845672,BAD REQUEST,1251868453,BAD REQUEST,False,0,0,2,2
25484,change,234810735,/* History */,59.180.151.222,1,1,0,20080828164140,68.50.79.137,NaN,...,False,1219941700,"""In the past, droughts have periodically led t...",1219280292,"""In the past, droughts have periodically led t...",True,0,0,58,53
25485,change,329132348,/* In Tamil Nadu */,66.184.61.179,1,1,0,20091201230141,RJFJR,NaN,...,False,1259708501,"""[[Tamil Nadu]] is home to one of the oldest a...",1259002677,"""[[Tamil Nadu]] is home to one of the oldest a...",False,0,0,303,303
25486,change,240599711,/* Biography */,75.157.130.175,6,1,2,20080924030549,J.delanoy,NaN,...,False,1222225549,Yo mama,1222225537,"== Biography ==,""Paulo Coelho was born in [[Ri...",True,11,1,2,205


In [15]:
train

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,num_recent_edits,num_recent_reversions,current_minor,current_timestamp,added_lines,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person
0,change,329595189,NaN,Nryan30,66,13,0,1259891940,219.78.124.42,NaN,...,0,0,False,1259894598,",==Leadership Through Emotion==,""Leadership is...",1259856305,NaN,False,1,0
1,change,232199357,/* Penis */,89.242.200.212,4,2,2,20080815230001,66.75.235.255,NaN,...,0,0,False,1218841201,It's fucking big.,1218816231,"""Falcon gained media attention after appearing...",True,4,1
2,change,329877752,Reverted edits by [[Special:Contributions/71.2...,Chamal N,18697,0,2,1208605428,71.208.113.72,NaN,...,2,1,True,1260025124,"""Japanese modern drama in the early twentieth ...",1260025104,""".nmbhgsdj;kfhds;akjfhds;fkjhsf;kjdshf;sdkjhfd...",False,3,0
3,change,253129486,NaN,Animaldudeyay1009,3,1,2,1227241317,J.delanoy,NaN,...,0,0,False,1227241840,I LIKE CHEESE. CHEESE IS GOOD. CHOCOLATE MILK ...,1227241120,"""A '''kaleidoscope''' is a tube of [[mirror]]s...",True,2,0
4,change,394520551,Adding Persondata using [[Project:AWB|AWB]] (7...,RjwilmsiBot,1602950,1309238,0,1257977968,LobãoV,NaN,...,0,0,True,1288757547,{{Persondata <!-- Metadata: see [[Wikipedia:Pe...,1285262356,",",False,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25483,change,327192485,http://fr.wikipedia.org/wiki/Championnat_de_Gu...,91.210.204.254,3,2,0,20091121232112,SmackBot,NaN,...,0,0,False,1258845672,BAD REQUEST,1251868453,BAD REQUEST,False,0,0
25484,change,234810735,/* History */,59.180.151.222,1,1,0,20080828164140,68.50.79.137,NaN,...,0,0,False,1219941700,"""In the past, droughts have periodically led t...",1219280292,"""In the past, droughts have periodically led t...",True,0,0
25485,change,329132348,/* In Tamil Nadu */,66.184.61.179,1,1,0,20091201230141,RJFJR,NaN,...,0,0,False,1259708501,"""[[Tamil Nadu]] is home to one of the oldest a...",1259002677,"""[[Tamil Nadu]] is home to one of the oldest a...",False,0,0
25486,change,240599711,/* Biography */,75.157.130.175,6,1,2,20080924030549,J.delanoy,NaN,...,2,1,False,1222225549,Yo mama,1222225537,"== Biography ==,""Paulo Coelho was born in [[Ri...",True,11,1


In [16]:
comment_empty.fit_transform(train)

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,num_recent_reversions,current_minor,current_timestamp,added_lines,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person,comment_empty
0,change,329595189,NaN,Nryan30,66,13,0,1259891940,219.78.124.42,NaN,...,0,False,1259894598,",==Leadership Through Emotion==,""Leadership is...",1259856305,NaN,False,1,0,True
1,change,232199357,/* Penis */,89.242.200.212,4,2,2,20080815230001,66.75.235.255,NaN,...,0,False,1218841201,It's fucking big.,1218816231,"""Falcon gained media attention after appearing...",True,4,1,False
2,change,329877752,Reverted edits by [[Special:Contributions/71.2...,Chamal N,18697,0,2,1208605428,71.208.113.72,NaN,...,1,True,1260025124,"""Japanese modern drama in the early twentieth ...",1260025104,""".nmbhgsdj;kfhds;akjfhds;fkjhsf;kjdshf;sdkjhfd...",False,3,0,False
3,change,253129486,NaN,Animaldudeyay1009,3,1,2,1227241317,J.delanoy,NaN,...,0,False,1227241840,I LIKE CHEESE. CHEESE IS GOOD. CHOCOLATE MILK ...,1227241120,"""A '''kaleidoscope''' is a tube of [[mirror]]s...",True,2,0,True
4,change,394520551,Adding Persondata using [[Project:AWB|AWB]] (7...,RjwilmsiBot,1602950,1309238,0,1257977968,LobãoV,NaN,...,0,True,1288757547,{{Persondata <!-- Metadata: see [[Wikipedia:Pe...,1285262356,",",False,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25483,change,327192485,http://fr.wikipedia.org/wiki/Championnat_de_Gu...,91.210.204.254,3,2,0,20091121232112,SmackBot,NaN,...,0,False,1258845672,BAD REQUEST,1251868453,BAD REQUEST,False,0,0,False
25484,change,234810735,/* History */,59.180.151.222,1,1,0,20080828164140,68.50.79.137,NaN,...,0,False,1219941700,"""In the past, droughts have periodically led t...",1219280292,"""In the past, droughts have periodically led t...",True,0,0,False
25485,change,329132348,/* In Tamil Nadu */,66.184.61.179,1,1,0,20091201230141,RJFJR,NaN,...,0,False,1259708501,"""[[Tamil Nadu]] is home to one of the oldest a...",1259002677,"""[[Tamil Nadu]] is home to one of the oldest a...",False,0,0,False
25486,change,240599711,/* Biography */,75.157.130.175,6,1,2,20080924030549,J.delanoy,NaN,...,1,False,1222225549,Yo mama,1222225537,"== Biography ==,""Paulo Coelho was born in [[Ri...",True,11,1,False


In [17]:
train

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,num_recent_edits,num_recent_reversions,current_minor,current_timestamp,added_lines,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person
0,change,329595189,NaN,Nryan30,66,13,0,1259891940,219.78.124.42,NaN,...,0,0,False,1259894598,",==Leadership Through Emotion==,""Leadership is...",1259856305,NaN,False,1,0
1,change,232199357,/* Penis */,89.242.200.212,4,2,2,20080815230001,66.75.235.255,NaN,...,0,0,False,1218841201,It's fucking big.,1218816231,"""Falcon gained media attention after appearing...",True,4,1
2,change,329877752,Reverted edits by [[Special:Contributions/71.2...,Chamal N,18697,0,2,1208605428,71.208.113.72,NaN,...,2,1,True,1260025124,"""Japanese modern drama in the early twentieth ...",1260025104,""".nmbhgsdj;kfhds;akjfhds;fkjhsf;kjdshf;sdkjhfd...",False,3,0
3,change,253129486,NaN,Animaldudeyay1009,3,1,2,1227241317,J.delanoy,NaN,...,0,0,False,1227241840,I LIKE CHEESE. CHEESE IS GOOD. CHOCOLATE MILK ...,1227241120,"""A '''kaleidoscope''' is a tube of [[mirror]]s...",True,2,0
4,change,394520551,Adding Persondata using [[Project:AWB|AWB]] (7...,RjwilmsiBot,1602950,1309238,0,1257977968,LobãoV,NaN,...,0,0,True,1288757547,{{Persondata <!-- Metadata: see [[Wikipedia:Pe...,1285262356,",",False,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25483,change,327192485,http://fr.wikipedia.org/wiki/Championnat_de_Gu...,91.210.204.254,3,2,0,20091121232112,SmackBot,NaN,...,0,0,False,1258845672,BAD REQUEST,1251868453,BAD REQUEST,False,0,0
25484,change,234810735,/* History */,59.180.151.222,1,1,0,20080828164140,68.50.79.137,NaN,...,0,0,False,1219941700,"""In the past, droughts have periodically led t...",1219280292,"""In the past, droughts have periodically led t...",True,0,0
25485,change,329132348,/* In Tamil Nadu */,66.184.61.179,1,1,0,20091201230141,RJFJR,NaN,...,0,0,False,1259708501,"""[[Tamil Nadu]] is home to one of the oldest a...",1259002677,"""[[Tamil Nadu]] is home to one of the oldest a...",False,0,0
25486,change,240599711,/* Biography */,75.157.130.175,6,1,2,20080924030549,J.delanoy,NaN,...,2,1,False,1222225549,Yo mama,1222225537,"== Biography ==,""Paulo Coelho was born in [[Ri...",True,11,1


In [18]:
is_IP = FunctionTransformer(_is_IP)
is_IP.fit_transform(train)

,EditType,EditID,comment,user,user_edit_count,user_distinct_pages,user_warns,user_reg_time,prev_user,common,...,num_recent_reversions,current_minor,current_timestamp,added_lines,previous_timestamp,deleted_lines,isvandalism,num_edits_5d_before,is_person,is_IP
0,change,329595189,NaN,Nryan30,66,13,0,1259891940,219.78.124.42,NaN,...,0,False,1259894598,",==Leadership Through Emotion==,""Leadership is...",1259856305,NaN,False,1,0,False
1,change,232199357,/* Penis */,89.242.200.212,4,2,2,20080815230001,66.75.235.255,NaN,...,0,False,1218841201,It's fucking big.,1218816231,"""Falcon gained media attention after appearing...",True,4,1,True
2,change,329877752,Reverted edits by [[Special:Contributions/71.2...,Chamal N,18697,0,2,1208605428,71.208.113.72,NaN,...,1,True,1260025124,"""Japanese modern drama in the early twentieth ...",1260025104,""".nmbhgsdj;kfhds;akjfhds;fkjhsf;kjdshf;sdkjhfd...",False,3,0,False
3,change,253129486,NaN,Animaldudeyay1009,3,1,2,1227241317,J.delanoy,NaN,...,0,False,1227241840,I LIKE CHEESE. CHEESE IS GOOD. CHOCOLATE MILK ...,1227241120,"""A '''kaleidoscope''' is a tube of [[mirror]]s...",True,2,0,False
4,change,394520551,Adding Persondata using [[Project:AWB|AWB]] (7...,RjwilmsiBot,1602950,1309238,0,1257977968,LobãoV,NaN,...,0,True,1288757547,{{Persondata <!-- Metadata: see [[Wikipedia:Pe...,1285262356,",",False,0,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25483,change,327192485,http://fr.wikipedia.org/wiki/Championnat_de_Gu...,91.210.204.254,3,2,0,20091121232112,SmackBot,NaN,...,0,False,1258845672,BAD REQUEST,1251868453,BAD REQUEST,False,0,0,True
25484,change,234810735,/* History */,59.180.151.222,1,1,0,20080828164140,68.50.79.137,NaN,...,0,False,1219941700,"""In the past, droughts have periodically led t...",1219280292,"""In the past, droughts have periodically led t...",True,0,0,True
25485,change,329132348,/* In Tamil Nadu */,66.184.61.179,1,1,0,20091201230141,RJFJR,NaN,...,0,False,1259708501,"""[[Tamil Nadu]] is home to one of the oldest a...",1259002677,"""[[Tamil Nadu]] is home to one of the oldest a...",False,0,0,True
25486,change,240599711,/* Biography */,75.157.130.175,6,1,2,20080924030549,J.delanoy,NaN,...,1,False,1222225549,Yo mama,1222225537,"== Biography ==,""Paulo Coelho was born in [[Ri...",True,11,1,True


## Define cleaning and feature-generation transformers

In [55]:
bad_request_replacer = FunctionTransformer(
    lambda X: X.replace(to_replace="BAD REQUEST", value=nan, inplace=False)
)
word_counter = FunctionTransformer(_word_counter)
comment_empty = FunctionTransformer(_comment_empty)
is_IP = FunctionTransformer(_is_IP)